# Experiment 1

## Setup

* Processor: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz 1.80 GHz
* Number of cores: 4
* Memory: 16.0 GB

## Dataset

* The r/place Parquet dataset - 12GB (22GB uncompressed) - /datas/2022_place_deephaven.parquet

## Tests

* Reading in the file, speed comparison
* Compute metrics of a column: (mean, std)
* Finding the unique value of a column
* Cumulative sum of a column
* Groupby Aggregation

## Dask

### A low-level sceduler and a high-level partial Pandas replacement, geared toward running code on compute clusters

## Modin

### A drop-in replacement for Pandas, powered by either Dask or Ray

In [ ]:
import ray
import modin.pandas as pd
ray.init()

In [ ]:
start_time = time.time()
df = pd.read_parquet(inputpath)
print(f"1. Modin - Reading exec time: {time.time() - start_time}")

In [ ]:
start_time = time.time()
_ = df['x1'].mean()
print(f"2. Modin - Mean exec time: {time.time() - start_time}")

In [ ]:
start_time = time.time()
_ = df['x1'].std()
print(f"3. Modin - Std exec time: {time.time() - start_time}")

In [ ]:
start_time = time.time()
_ = df['user_id'].unique()
print(f"4. Modin - Unique exec time: {time.time() - start_time}")

In [ ]:
start_time = time.time()
_ = df['y1'].cumsum()
print(f"5. Modin - Cumsum exec time: {time.time() - start_time}")

In [ ]:
start_time = time.time()
_ = df.groupby('user_id')['x1'].mean()
print(f"6. Modin - Groupby Aggregation exec time: {time.time() - start_time}")

## Results

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tests = ['read', 'mean', 'std', 'unique', 'cumsum', 'groupby']
results = { 'pandas' : [60 + 46, 0.088, 1.13, 60+17, 1.09, 120 +2],
            'dask' : [17, 0.673, 4.33, 29.5, 4.12, 60+22],
            'modin' : [3*60+45, 0.526, 5.75, 120+47, 0.276, 480+44],
            'vaex': [120+43, 5.72, 5.59, 480+44, 7.26, 6*60 + 48]
        } 

results_df = pd.DataFrame(data=results, index=tests)

for test in results_df.index:
	results_df.loc[test].plot(kind='bar', title=test)
	plt.show()

AttributeError: module 'pandas.plotting._matplotlib' has no attribute 'plot'